In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from tqdm.notebook import tqdm

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

TESTSAMPLEPATH = '../input/cassava-leaf-disease-classification/sample_submission.csv'
CLASSESJSONPATH = '../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'
TESTDATAPATH = '../input/cassava-leaf-disease-classification/test_images'
TRAINDATAPATH = '../input/cassava-leaf-disease-classification/train_images'
cfg = {}
cfg['image_size'] = 512 # (600, 800)
cfg['device'] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg['batch_size'] = 16
classes = {}
num_class = 0
# get the class
import json

with open(CLASSESJSONPATH) as f:
    classes = json.load(f)
    num_class = len(classes)

def readImage(ID, path):
    ID = ID.split('.')[0]
    filepath = os.path.join(path, ID + '.jpg')
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# dataset and data loader
class CLCdataset(Dataset):
    def __init__(self, df,isTrain = True):
        self.df = df
        self.isTrain = isTrain
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        ID = self.df['image_id'].iloc[idx].split('.')[0]
        img = readImage(ID, TRAINDATAPATH if self.isTrain else TESTDATAPATH)
        if self.isTrain:
            img = train_transform(image = img)['image']
        else:
            img = test_transform(image = img)['image']
        if self.isTrain:
            label = self.df['label'].iloc[idx]
            return ID, img, label
        return ID, img
    

In [ ]:
test_transform = A.Compose([
    A.Resize(cfg['image_size'], cfg['image_size'],p=1),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2() # (H,W,C) to (C,H,W)
])

In [ ]:
class CFCmodel(nn.Module):
    def __init__(self, out_dim):
        super(CFCmodel, self).__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=False)
        self.model.classifier = nn.Linear(in_features=1536,out_features=out_dim, bias=True)
        self.model.eval()
#         self.classifier = nn.Sequential(nn.Linear(1000, out_dim),
#                                         nn.Softmax())
#         self.classifier = nn.Sequential(nn.Linear(1000, out_dim))
        # freeze attibute
#         for p in self.features.parameters():
#             p.requires_grad = False
    def forward(self, input):
        return self.model(input)

In [ ]:

def test(model, dataloader):
    model.eval()
    pred_list = []
    pred_id_list = []
    with torch.no_grad():
        with tqdm(dataloader,unit='batch',desc='Test') as tqdm_loader:
            for idx, (ID, img) in enumerate(tqdm_loader):

                img = img.to(device=cfg['device'])
                pred = model(img).detach().cpu().argmax(dim=1)

                pred_list.append(pred)
                pred_id_list.append(ID)
    pred_list = np.concatenate(pred_list,axis=0)
    pred_id_list =  np.concatenate(pred_id_list,axis=0)
    return pred_list, pred_id_list

test_df = pd.read_csv(TESTSAMPLEPATH)
test_dataset = CLCdataset(test_df,isTrain=False)
test_dataloader = DataLoader(test_dataset, batch_size=cfg['batch_size'], shuffle=False, num_workers=2)


model = CFCmodel(num_class).to(cfg['device'])

model.load_state_dict(torch.load('../input/k/moore0403/cfdc-competition/model0_6.pth'))

pred_list, pred_id_list = test(model, test_dataloader)

pred_id_list = [ID +'.jpg' for ID in pred_id_list]

submit_df  = pd.DataFrame(columns=test_df.columns)
submit_df['image_id'] = pred_id_list
submit_df['label']=pred_list
submit_df.to_csv(os.path.join('./', 'submission.csv'),index=False)
# print(submit_df.head(10))